###SENDING A TEST EVENT TO THE EVENT HUB

In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json
#Event hub configuration
# getting secret value from key vault
eventhub_connection_string = dbutils.secrets.get(scope = "key-vault-scope", key ="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"  

producer = EventHubProducerClient.from_connection_string(
    conn_str=eventhub_connection_string,
    eventhub_name=EVENT_HUB_NAME
)

def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

event = {
    "id": 222,
    "name": "hamza",
    "age": 42
}

send_event(event)
producer.close()


### API TESTING

In [0]:
import requests
import json
weatherapikey= dbutils.secrets.get(scope = "key-vault-scope", key ="weather-api-key")
location = "Montréal"
base_url = "http://api.weatherapi.com/v1"
current_url = f"{base_url}/current.json"
params = {
    "key": weatherapikey,
    "q": location
}
response = requests.get(current_url, params=params)
if response.status_code == 200:
    data = response.json()
    print("Current weather for", location, ":")
    print(json.dumps(data, indent=4, ensure_ascii=False))
else:
    print("Error:", response.status_code)


Current weather for Montréal :
{
    "location": {
        "name": "Montréal",
        "region": "Quebec",
        "country": "Canada",
        "lat": 45.5,
        "lon": -73.5833,
        "tz_id": "America/Toronto",
        "localtime_epoch": 1749845704,
        "localtime": "2025-06-13 16:15"
    },
    "current": {
        "last_updated_epoch": 1749845700,
        "last_updated": "2025-06-13 16:15",
        "temp_c": 17.4,
        "temp_f": 63.3,
        "is_day": 1,
        "condition": {
            "text": "Partly Cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 7.2,
        "wind_kph": 11.5,
        "wind_degree": 204,
        "wind_dir": "SSW",
        "pressure_mb": 1022.0,
        "pressure_in": 30.17,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 42,
        "cloud": 0,
        "feelslike_c": 17.4,
        "feelslike_f": 63.3,
        "windchill_c": 18.6,
        

### sending the complete weather data to event HUB in every 1 min

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime, timedelta

eventhub_connection_string = dbutils.secrets.get(scope = "key-vault-scope", key ="eventhub-connection-string")
EVENT_HUB_NAME = "weatherstreamingeventhub"  

producer = EventHubProducerClient.from_connection_string(
    conn_str=eventhub_connection_string,
    eventhub_name=EVENT_HUB_NAME
)
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event, ensure_ascii=False)))
    producer.send_batch(event_data_batch)

def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Montréal" 
    weatherapikey= dbutils.secrets.get(scope = "key-vault-scope", key ="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    return merged_data 
last_sent_time = datetime.now() - timedelta(seconds=60)
# Main program
def process_batch(batch_df, batch_id):
    global  last_sent_time
    try:    
        current_time = datetime.now()
        if (current_time - last_sent_time).total_seconds() >= 60: 
        # Fetch weather data
            weather_data = fetch_weather_data()
        
        # Send the weather data (current weather part)
            send_event(weather_data)
            last_sent_time = current_time
            print(f'Event Sent at {last_sent_time}')

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()



Event Sent at 2025-06-13 21:52:44.606736


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can